In [ ]:
!pip install langchain-groq
!pip install flask-cors
!pip install langchain
!pip install langchain-groq
!pip install chromadb
!pip install langchain_community beautifulsoup4

!pip install Flask pyngrok

!ngrok authtoken {......write_your_authtoken_here....}


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from threading import Thread
from pyngrok import ngrok
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_community.document_loaders import WebBaseLoader
import pandas as pd
import chromadb
import uuid

app = Flask(__name__)
CORS(app)

@app.route('/query', methods=['POST'])
def handle_query():
    data = request.json
    query = data.get('query')
    username = data.get('username')  # Get the username from the request

    if not query or not username:
        return jsonify({"error": "Both URL and username are required."}), 400

    # Initialize the ChatGroq model
    llm = ChatGroq(
        temperature=0,
        model_name="llama-3.1-70b-versatile",
        groq_api_key="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"
    )

    # Load job posting data from the URL
    loader = WebBaseLoader(query)
    page_data = loader.load().pop().page_content

    # Create a prompt template to extract job postings
    prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTIONS:
        The scrapped text is from a career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: 'role', 'experience', 'skills', and 'description'.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLES):
        """
    )

    # Extract job postings using the LLM
    chain_extract = prompt_extract | llm
    res = chain_extract.invoke(input={'page_data': page_data})

    # Parse the JSON response
    json_parser = JsonOutputParser()
    json_res = json_parser.parse(res.content)

    # Load the portfolio data from CSV
    df = pd.read_csv("my_portfolio.csv")

    # Initialize the ChromaDB client
    client = chromadb.PersistentClient('vectorstore')
    collection = client.get_or_create_collection(name="portfolio")

    # Add documents to the collection if it's empty
    if not collection.count():
        for _, row in df.iterrows():
            collection.add(documents=row["Techstack"],
                          metadatas={"links": row["Links"]},
                          ids=[str(uuid.uuid4())])

    # Loop through each job posting
    for job in json_res:
        skills = job.get('skills', [])
        client_name = "Hiring Manager"
        if skills:
            query_texts = [skill.strip() for skill in skills]
            links = collection.query(query_texts=query_texts, n_results=2)

            # Create a prompt template for the email, including the username
            prompt_email = PromptTemplate.from_template(
                f"""
                ### JOB DESCRIPTION:
                {{job_description}}

                ### INSTRUCTION:
                You are {username}, a dedicated third-year Computer Engineering student.
                You actively engage in projects that enhance your technical skills and have good communication skills.
                Your task is to write a cold email to {{client_name}} regarding the job mentioned above, showcasing your capabilities and how your academic pursuits align with their needs.
                Highlight your interests in innovative solutions and user engagement, as well as your commitment to creating impactful projects.
                Also, add the most relevant ones from the following links to showcase your portfolio: {{link_list}}
                Remember you are {username}, a Computer Engineering Student.
                Maintain a professional tone and avoid providing a preamble.
                ### EMAIL (NO PREAMBLE):
                """
            )

            # Generate the email content
            chain_email = prompt_email | llm
            res = chain_email.invoke({"job_description": str(job), "link_list": links, "client_name": client_name})

            # Structure the response
            response = res.content

            if not page_data:
                return jsonify({"error": "No data found at the provided URL."}), 400
            else:
                return jsonify({"response": response})


def run():
    app.run(port=5018)

if __name__ == '__main__':
    Thread(target=run).start()
    public_url = ngrok.connect(5018)
    print("Public URL:", public_url)
